In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from models import SRN2VecModel
import json

In [2]:
city = "hanover"

In [3]:
network = RoadNetwork()
network.load(f"../../osm_data/{city}")

In [4]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network)

In [5]:
model.generate_data(n_shortest_paths=100, number_negative=3, window_size=900, save_batch_size=100, city=city)

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
%lprun -f model.extract_pairs model.generate_train_pairs(network, paths, 900, 7)

In [3]:
with open(f"srn2vec-traindata-{city}.json", "r") as fp:
    a = json.load(fp)

In [7]:
"""
Build and train model
"""
network = RoadNetwork()
network.load(f"../../osm_data/{city}")
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model = SRN2VecModel(None, device, network, remove_highway_label=True)
model.load_dataset(f"./srn2vec-traindata-{city}.json")

In [8]:
model.train(epochs=5, batch_size=256)

Epoch: 0, Iteration: 0, sample_loss: 0.70181804895401, Avg. Loss: 0.70181804895401
Epoch: 0, Iteration: 1000, sample_loss: 0.42624327540397644, Avg. Loss: 0.5330494427002155
Epoch: 0, Iteration: 2000, sample_loss: 0.3482593595981598, Avg. Loss: 0.4579063818372529
Epoch: 0, Iteration: 3000, sample_loss: 0.3280641436576843, Avg. Loss: 0.41688991114819934
Epoch: 0, Iteration: 4000, sample_loss: 0.3461648225784302, Avg. Loss: 0.39428554661152987
Epoch: 0, Iteration: 5000, sample_loss: 0.2526435852050781, Avg. Loss: 0.3799369432608668
Epoch: 0, Iteration: 6000, sample_loss: 0.34531858563423157, Avg. Loss: 0.3704917191306942
Epoch: 0, Iteration: 7000, sample_loss: 0.3511354923248291, Avg. Loss: 0.36375156377846984
Epoch: 0, Iteration: 8000, sample_loss: 0.31269240379333496, Avg. Loss: 0.35869034763284513
Epoch: 0, Iteration: 9000, sample_loss: 0.3508126139640808, Avg. Loss: 0.3545463475359505
Epoch: 0, Iteration: 10000, sample_loss: 0.3209928870201111, Avg. Loss: 0.3514116883751703
Epoch: 0,

In [9]:
model.save_model(path="../model_states/srn2vec/")